# Refinements

## Setup

In [ ]:
import json
import os
from tqdm import tqdm
from colorama import Style,Fore

In [ ]:
def openJson(path):
    
    # Creates the file if not existing
    if not os.path.exists(path):
        with open(path, "w", encoding="utf-8") as file:
            json.dump([], file)
            
    # Open it otherwise
    with open(path, "r", encoding="utf-8") as file:
        data = json.load(file)
    return data

def saveJson(path,data):
    with open(path, "w", encoding="utf-8") as f:
       json.dump(data, f, ensure_ascii=False, indent=2)
       #print(Style.BRIGHT+Fore.GREEN+'\n json saved'+Style.RESET_ALL)

## Language Detection

In [ ]:
from langdetect import detect

In [ ]:
def languageDetection(videosPath,savePath):
    videos = openJson(videosPath)
        
    for video in tqdm(videos,'Language Detection...'):
        try:
            lg = detect((video['titre_video']+video['description']).lower())
            if lg == 'fr':
                video['langue'] = 'fr'
            else:
                video['langue'] = 'autre'
        except:
            print(Style.BRIGHT+Fore.YELLOW+f"\n Detection failed for video : {video['id_video']}"+Style.RESET_ALL)
            
    saveJson(savePath,videos)
  

In [ ]:
languageDetection('../../collecting/jsons/videos.json','./jsons/videosR1.json')

## Add channel location

In [ ]:
def locationAdding(channelsPath,videosPath,savePath):
    videosR1 = openJson(videosPath)
    channels = openJson(channelsPath)
    
    ################### Channels Countries dictionary
    channels_countries = {}
    for channel in channels:
        channels_countries[channel['id_chaine']]= channel['localisation']
     
    ################## Refine the language with the country Code if existent
    for video in tqdm(videosR1,'locationAdding...'):
        if video['langue'] == 'fr':  
            channelId = video['id_chaine']
            try:
                country = channels_countries[channelId]
                if country:
                    video['langue']+=f'-{country}'
            except:
                print(Style.BRIGHT+Fore.YELLOW+f"\n probleme while locationAdding video: {video['id_video']}"+Style.RESET_ALL)
                
    ################## Save the New json file
    saveJson(savePath,videosR1)

In [ ]:
locationAdding('../../collecting/jsons/channels.json','./jsons/videosR1.json','./jsons/videosR2.json')

## Detect Channel Location

- We still have french videos but we are not sure if it's is fr-France, because channel has unknowen location
- The idea is trying to detect a channel location to be able to decide about the video.

# Filter by Language

In [ ]:
import json
import os
from tqdm import tqdm
from colorama import Style,Fore

# Lundi 10H